#Taller 4: Neural Networks and Deep Learning
#$Integrantes$:
>Juan Felipe Baquero Vargas

>Lina Fernanda Rosales Castro

## A) Download the code and run the files

In [7]:
!dir

LICENSE		     linear_decoder_exercise.py  softmax_exercise.py
README.md	     load_MNIST.py		 sparse_autoencoder.py
cnn.py		     load_MNIST.pyc		 sparse_autoencoder.pyc
cnn_exercise.py      load_images.py		 stacked_ae_exercise.py
data		     output			 stacked_autoencoder.py
display_network.py   pca_gen.py			 stacked_autoencoder.pyc
display_network.pyc  sample_images.py		 stl_exercise.py
gradient.py	     sample_images.pyc		 train.py
gradient.pyc	     softmax.py
iterate.dat	     softmax.pyc


In [8]:
cd ufldl_tutorial/

[Errno 2] No such file or directory: 'ufldl_tutorial/'
/home/jfbaquerov/MachineLearning/ufldl_tutorial


In [10]:
%%bash
python softmax_exercise.py

  File "softmax_exercise.py", line 69
    print num_grad, grad
                 ^
SyntaxError: invalid syntax


In [11]:
%%bash
python train.py

  File "train.py", line 78
    print cost, grad
             ^
SyntaxError: invalid syntax


Ambos codigos presentaron problemas al correr desde el notebook de python asi que se corrio directamente desde consola, se adjuntas imagenes de la ejecucion y resultados

In [13]:
cd ..

/home/jfbaquerov/MachineLearning


Imagen en la que se inicia  la ejecucion de train.py
<img src="imagenes_entrega/Punto3_1.PNG">

Cuando termino la ejecucion por exceder el numero de iteraciones:
<img src="imagenes_entrega/Punto3_2.PNG">

El resumen de la ejecucion:
<img src="imagenes_entrega/Punto3_3.PNG">

Los resultados obtenidos fueron:
<img src="imagenes_entrega/weights.png">

Al ejecutar softmax_exercise
<img src="imagenes_entrega/Punto3_sm1.PNG">

El fin de la ejecucion por exceder el numero maximo de iteraciones:
<img src="imagenes_entrega/Punto3_sm2.PNG">

Al final softMax Presento un Accuracy de **92.31%**

##B) Modify the code to train

In [19]:
cd ufldl_tutorial/

/home/jfbaquerov/MachineLearning/ufldl_tutorial


In [20]:
!ls

LICENSE		     linear_decoder_exercise.py  softmax_exercise.py
README.md	     load_MNIST.py		 sparse_autoencoder.py
cnn.py		     load_MNIST.pyc		 sparse_autoencoder.pyc
cnn_exercise.py      load_images.py		 stacked_ae_exercise.py
data		     output			 stacked_autoencoder.py
display_network.py   pca_gen.py			 stacked_autoencoder.pyc
display_network.pyc  sample_images.py		 stl_exercise.py
gradient.py	     sample_images.pyc		 train.py
gradient.pyc	     softmax.py			 weights.png
iterate.dat	     softmax.pyc


Se realiza una copia de train.py llamada train2.py, la cual sera la modificacion agregando la funcion de costo RICA

In [21]:
!ls

LICENSE		     linear_decoder_exercise.py  softmax_exercise.py
README.md	     load_MNIST.py		 sparse_autoencoder.py
cnn.py		     load_MNIST.pyc		 sparse_autoencoder.pyc
cnn_exercise.py      load_images.py		 stacked_ae_exercise.py
data		     output			 stacked_autoencoder.py
display_network.py   pca_gen.py			 stacked_autoencoder.pyc
display_network.pyc  sample_images.py		 stl_exercise.py
gradient.py	     sample_images.pyc		 train.py
gradient.pyc	     softmax.py			 train2.py
iterate.dat	     softmax.pyc		 weights.png


In [22]:
!cat train.py

import numpy as np
import scipy.optimize
import sample_images
import sparse_autoencoder
import gradient
import display_network
import load_MNIST


##======================================================================
## STEP 0: Here we provide the relevant parameters values that will
#  allow your sparse autoencoder to get good filters; you do not need to
#  change the parameters below.

# number of input units
visible_size = 28 * 28
# number of input units
hidden_size = 196

# desired average activation of the hidden units.
# (This was denoted by the Greek alphabet rho, which looks like a lower-case "p",
#  in the lecture notes).
sparsity_param = 0.1
# weight decay parameter
lambda_ = 3e-3
# weight of sparsity penalty term
beta = 3
# debug
debug = False


##======================================================================
## STEP 1: Implement sampleIMAGES
#
#  After implementing sampleIMAGES, the display_network command should
#  display a random sample of 200 patches from the

En el archivo de train se modificara el **Paso 4** cambiando el calculo del costo del sparse_autoencoder por el calculo propuesto por el enunciado del ejercicio

$min_W(\lambda*||Wx||_1+1/2*||W^tWx-x||^2_2)$

Para hacer esto se realiza el cambio en:
>J = lambda x: sparse_autoencoder.sparse_autoencoder_cost(x, visible_size, hidden_size,lambda_, sparsity_param,beta, patches)

por:
>J = lambda x: RICA_cost(x, visible_size, hidden_size,lambda_, sparsity_param,beta, patches)
                                                         
                                                         

Donde se definio Rica cost como se muestra acontinuacion:

In [24]:
def RICA_cost(theta, visible_size, hidden_size, lambda_,
              sparsity_param,beta, data):
    #Lo siguiente es la misma tranformacion de datos que realizan en el sparse_autoencoder_cost
    
    # The input theta is a vector (because minFunc expects the parameters to be a vector).
    # We first convert theta to the (W1, W2, b1, b2) matrix/vector format, so that this
    # follows the notation convention of the lecture notes.

    W1 = theta[0:hidden_size * visible_size].reshape(hidden_size, visible_size)
    W2 = theta[hidden_size * visible_size:2 * hidden_size * visible_size].reshape(visible_size, hidden_size)
    b1 = theta[2 * hidden_size * visible_size:2 * hidden_size * visible_size + hidden_size]
    b2 = theta[2 * hidden_size * visible_size + hidden_size:]
    
    # Number of training examples
    m = data.shape[1]

    eps =  np.tile(b1, (m, 1)).transpose()
    # Forward propagation
    z2 = W1.dot(data)**2 
    #a2 = sigmoid(z2)
    z3 = W2.dot(a2) + np.tile(b2, (m, 1)).transpose()
    #h = sigmoid(z3)
    wx = np.sqrt( np.sum(z2 +eps) )
    wTwx = 0.5*np.sum( W.T.dot( W.dot(x) )-x + eps )
    
    cost = (lambda_*wx+wTwx)/m
    
    gradient = lambda_ * np.dot(np.dot(W, x) / wx, x.T)
    grad = (gradient + np.dot(np.dot(W, wTwx), x.T) + np.dot(np.dot(W, x), wTwx.T) ) / m
    
    return cost, grad
    

Se crea un nuevo train que es la copia del anterior pero con los cambios comentados, acontinucacion se muestra el archivo:

In [26]:
!cat train2.py

import numpy as np
import scipy.optimize
import sample_images
import sparse_autoencoder
import gradient
import display_network
import load_MNIST


##======================================================================
## STEP -1: We create a RICA cost function
def RICA_cost(theta, visible_size, hidden_size, lambda_,
              sparsity_param,beta, data):
    #Lo siguiente es la misma tranformacion de datos que realizan en el sparse_autoencoder_cost
    
    # The input theta is a vector (because minFunc expects the parameters to be a vector).
    # We first convert theta to the (W1, W2, b1, b2) matrix/vector format, so that this
    # follows the notation convention of the lecture notes.

    W1 = theta[0:hidden_size * visible_size].reshape(hidden_size, visible_size)
    W2 = theta[hidden_size * visible_size:2 * hidden_size * visible_size].reshape(visible_size, hidden_size)
    b1 = theta[2 * hidden_size * visible_size:2 * hidden_size * visible_size + hidden_size]
    b2 = theta[2

Los resultados Luego del cambio fueron:
<img src="imagenes_entrega/weights_22.png">

Al ejecutar softmax_excersise con el nuevo costo los resultados fueron:
<img src="imagenes_entrega/punto3_C1.PNG">

El cambio de la función en el primer ejercicio de Vectorizacion nos mostro unas imagenes un poco mas difusas, lo cual seria un mejor desempeño, ya que esto es lo podido en el ejercicio del tutorial.

Para el ejercicio de Sofmax_Regression, la mejora fue muy poca paso del **92.31** al **92.43** acercandose mas al objetivo de ***92.6***, el cambio de funcion objetivo acerco mas al accuracy que se buscaba en el taller.